In [1]:
!pip install timm


In [3]:
# 🧪 Codex Sandbox: EarthGuard Vision Model Refactor
# Project Goal: Improve inference speed, accuracy, and architectural robustness for object detection
# Context: Moving beyond YOLO if necessary. Model should be optimized for real-world litter detection from street/vehicle-mounted cameras.

# ✅ Setup
import torch
import torchvision
import timm  # PyTorch Image Models - for FastViT, ConvNeXt, etc.
from torchvision import transforms
from PIL import Image
import cv2

# 🧠 Placeholder: Load example dataset (replace with actual dataloader later)
def load_sample_image(path):
    image = Image.open(path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0)

# 🚀 FastViTHD from timm (if available)
def load_fastvit_model():
    model = timm.create_model("hf_hub:apple/fastvit_tiny", pretrained=True)  # Fallback to fast small model
    model.eval()
    return model

# 🔁 Inference loop
@torch.no_grad()
def run_inference(model, image_tensor):
    output = model(image_tensor)
    return output

# 🧪 Example run (replace 'sample.jpg' with real input)
if __name__ == "__main__":
    image_path = 'litter_survey_dataset/2024-07-21/RayBan/photo-21700_singular_display_fullPicture.jpeg'
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for Matplotlib
    
    model = load_fastvit_model()
    output = run_inference(model, image)
    print("Output shape:", output.shape)  # Will vary based on head used

# 💡 Future directions:
# - Swap backbone (FastViTHD, ConvNeXt, etc.)
# - Plug into Detectron2 or MMDetection for box prediction
# - Add custom classification head for 'litter vs non-litter'
# - Export to ONNX or TorchScript for mobile/embedded inference


RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-6829fc5a-4a4b69b33f72d6721d6e3dc8;1f0907f1-3359-44af-833b-90806ea1f184)

Repository Not Found for url: https://huggingface.co/apple/fastvit_tiny/resolve/main/config.json.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.